## Arnoldi's methods

In [169]:
import LinearAlgebra
const la = LinearAlgebra
MAX_ITER = 100;

In [170]:
A = [4. -1 0 -1 0 0;
     -1 4 -1 0 -1 0;
     0 -1 4. 0 0 -1;
     -1 0 0. 4 -1 0;
     0 -1 0 -1 4 -1;
     0 0 -1 0. -1 4]

b = [0, 5, 0, 6, -2, 6.]

x0 = zeros(size(b));
# n
v = [1., 0, 2, 0, 1, 1];

### Arnoldi Gram-Schmidt
Input: $A\in\mathbb{R}^{n\times n}=[a_1,...,a_n], v\in\mathbb{R}^{n}$ any vector

In [171]:
function arnoldi(A:: Matrix, v:: Vector, m:: Int64)
    # Input: nxn matrix A, nx1 column vector v, and integer m
    # Output: (m + 1)xm matrix H and nx1 a nx(m + 1) matrix V
    n = size(A, 1)
    V = zeros((n, m + 1))
    H = zeros((m + 1, m))
    V[:, 1] = v/la.norm(v)
    for j = 1:m
        for i = 1:j
            H[i, j] = la.dot(A*V[:, j], V[:, i])
        end
        w = A*V[:, j] - V[:, 1:j]*H[1:j, j]
        H[j+1, j] = la.norm(w)
        if H[j+1, j] < eps()
            break
        end
        V[:, j+1] = w/H[j+1, j]
    end
    return H, V
end

arnoldi (generic function with 1 method)

In [172]:
m = 3
H, V = arnoldi(A, v, m);
display(H)
display(V)

4×3 Matrix{Float64}:
 3.14286  1.92195  -2.77556e-15
 1.92195  4.6472    1.18932
 0.0      1.18932   3.95556
 0.0      0.0       0.812257

6×4 Matrix{Float64}:
 0.377964   0.168563    0.289595   -0.585252
 0.0       -0.786629    0.191845    0.228787
 0.755929   0.140469   -0.282287    0.128662
 0.0       -0.393314    0.0959226  -0.699892
 0.377964  -0.0280939   0.750937    0.313906
 0.377964  -0.421408   -0.475959    0.0140224

### Arnoldi-Modified Gram-Schmidt

In [173]:
function arnoldi_modified(A:: Matrix, v:: Vector, m:: Int64)
    # Input: nxn matrix A, nx1 column vector v, and integer m
    # Output: (m + 1)xm matrix H and nx1 a nx(m + 1) matrix V
    n = size(A, 1)
    
    H = zeros((m + 1, m))
    V = zeros((n, m + 1))
    
    V[:, 1] = v/la.norm(v)
    for j = 1:m
        w = A*V[:, j]
        for i = 1:j
            H[i, j] = la.dot(w, V[:, i])
            w = w - H[i, j]*V[:, i] 
        end
        H[j+1, j] = la.norm(w)
        if H[j+1, j] < eps()
            break
        end
        V[:, j+1] = w/H[j+1, j]
    end
    return H, V
end

arnoldi_modified (generic function with 1 method)

In [174]:
m = 3
H, V = arnoldi_modified(A, v, m)
display(H)
display(V)

4×3 Matrix{Float64}:
 3.14286  1.92195  -2.9976e-15
 1.92195  4.6472    1.18932
 0.0      1.18932   3.95556
 0.0      0.0       0.812257

6×4 Matrix{Float64}:
 0.377964   0.168563    0.289595   -0.585252
 0.0       -0.786629    0.191845    0.228787
 0.755929   0.140469   -0.282287    0.128662
 0.0       -0.393314    0.0959226  -0.699892
 0.377964  -0.0280939   0.750937    0.313906
 0.377964  -0.421408   -0.475959    0.0140224

### Householder Arnoldi

In [175]:
function householder_arnoldi(A:: Matrix, v:: Vector, m:: Int64)
    # Input: nxn matrix A, nx1 column vector v, and integer m < n
    # Output: nx(m+1) matrix H and an nxm orthonormal matrix V
    n = size(A, 1)
    I = 1.0 * Matrix(la.I, n, n)
    V = zeros((n, m + 1))
    H = zeros((n, m + 1))
    Z = zeros((n, m + 1))

    Z[:, 1] = v
    Q = I
    for j = 1:m + 1
        # w calculus
        val_sign = Z[j, j] >= 0.0 ? 1.0 : -1.0
        beta = val_sign * la.norm(Z[j:end, j])
        z = zeros(n)
        z[j] = beta - Z[j, j] + eps()
        for i = j + 1:n
            z[i] = -Z[i, j]
        end
        w = z / la.norm(z)
        # proyector
        P = I - 2 * w * w'
        # h_{j-1}
        H[:, j] = P * Z[:, j]
        # Pj*...*P2*P1
        Q = P * Q
        V[:, j] = Q'[:, j] # P1*P2...*Pj -> j-ésima
        if j <= m
            Z[:, j + 1] = Q * A * V[:, j]
        end
    end
    # delete the column 0 and return a (m+1)xm matrix H
    return H[1:m + 1, 2:m + 1], V
end

householder_arnoldi (generic function with 1 method)

In [176]:
m = 4
H, V = householder_arnoldi(A, v, m)
show(IOContext(stdout, :limit => false), "text/plain", H)
display(V)

5×4 Matrix{Float64}:
  3.14286      -1.92195       0.0           2.22045e-16
 -1.92195       4.6472        1.18932      -2.77556e-16
 -2.77556e-16   1.18932       3.95556      -0.812257
  2.77556e-16  -1.07499e-16  -0.812257      3.67305
 -5.55112e-17  -6.55942e-18   3.09757e-16   0.114091

6×5 Matrix{Float64}:
 0.377964  -0.168563   -0.289595   -0.585252    0.390312
 0.0        0.786629   -0.191845    0.228787    0.540433
 0.755929  -0.140469    0.282287    0.128662    0.2502
 0.0        0.393314   -0.0959226  -0.699892   -0.310248
 0.377964   0.0280939  -0.750937    0.313906   -0.440352
 0.377964   0.421408    0.475959    0.0140224  -0.45036

Se verifica $AV_m = V_{m+1}H$

In [177]:
isapprox(A*V[:, 1:m], V*H)

true

## Arnoldi’s Method for Linear Systems (FOM)

In [178]:
function FOM(A:: Matrix, b:: Vector, x0:: Vector, m:: Int64)
    r = b - A*x0
    beta = la.norm(r)
    v = r/beta
    H, V = arnoldi_modified(A, v, m)
    # delete last row of H and last column of V
    H = H[1:m, :]
    V = V[:, 1:m]
    return H, V, beta
end

FOM (generic function with 1 method)

In [179]:
m = 4
H, V, beta = FOM(A, b, x0, m)
y = la.inv(H)[:, 1] * beta
x = x0 + V*y

6-element Vector{Float64}:
 0.9999999999999999
 1.9999999999999998
 0.9999999999999999
 2.0
 0.9999999999999999
 2.0

In [180]:
function restarted_FOM(A:: Matrix, b:: Vector, x0:: Vector, m:: Int64, tol=1.0e-6)
    println("x[0] = $x0")
    for i=1:MAX_ITER
        H, V, beta = FOM(A, b, x0, m)
        y = la.inv(H)[:, 1] * beta
        x = x0 + V*y
        println("x[$i] = $x")
        if la.norm(x - x0) < tol
            break
        end
        # updating for next step
        x0 = copy(x)
    end
end

restarted_FOM (generic function with 2 methods)

In [181]:
m = 4
restarted_FOM(A, b, x0, m)

x[0] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
x[1] = [0.9999999999999999, 1.9999999999999998, 0.9999999999999999, 2.0, 0.9999999999999999, 2.0]
x[2] = [1.0, 2.0, 1.0, 2.0, 1.0000000000000002, 2.0]


## Incomplete Orthogonalization (IOM)

### Incomplete Arnoldi Process

In [182]:
function incomplete_arnoldi(A:: Matrix, v:: Vector, m:: Int64, k:: Int64)
    n = size(A, 1)
    V = zeros((n, m+1))
    H = zeros((m+1, m))
    V[:, 1] = v/la.norm(v)
    for j = 1:m
        w = A*V[:, j]
        for i = max(1, j - k + 1):j
            H[i, j] = la.dot(w, V[:, i])
            w = w - H[i, j]*V[:, i] 
        end
        H[j+1, j] = la.norm(w)
        V[:, j+1] = w/H[j+1, j]
    end
    return H, V
end

incomplete_arnoldi (generic function with 1 method)

In [194]:
function IOM(A:: Matrix, b:: Vector, x0:: Vector, m:: Int64, k:: Int64)
    r = b - A*x0
    beta = la.norm(r)
    v = r/beta
    H, V = incomplete_arnoldi(A, v, m, k)
    # delete last row of H and last column of V
    # H = H[1:m, :]
    # V = V[:, 1:m]
    return H, V, beta
end

IOM (generic function with 1 method)

In [195]:
m = 5
k = 3
H, V = IOM(A, b, x0, m, k)
display(H)
display(V)

6×5 Matrix{Float64}:
 4.67327  2.21877   3.747e-15   0.0          0.0
 2.21877  3.35573   0.690815   -4.07868e-14  0.0
 0.0      0.690815  4.32887     0.217999     2.94568
 0.0      0.0       0.217999    3.64213      0.0487366
 0.0      0.0       0.0         2.80489e-14  1.76082
 0.0      0.0       0.0         0.0          1.77754

6×6 Matrix{Float64}:
  0.0       -0.493309   -0.238968  -0.446699  -0.308737    0.295458
  0.497519  -0.0612751   0.7893    -0.354578   0.134578   -0.834818
  0.0       -0.493309   -0.238968  -0.446699  -0.308737    0.295458
  0.597022  -0.0914687  -0.272545   0.246814  -0.625391   -0.222677
 -0.199007  -0.702       0.337983   0.59444    0.0949961   0.171221
  0.597022  -0.0914687  -0.272545   0.246814  -0.625391   -0.222677

### DIOM
We need to keep only the $k$ previous $v_i$  in thr Arnoldi process, and we wish to be able to discard the others. However, if we implement the IOM algoritm, we still face the difficulty that when we compute the solution by formula $x_m = x_0 + V_my_m$, we will again need all the vectors $v_i$. One option would be ti recompute them at the end, but this will essentially double the cost of the algorithm. This raise the question of developing a formula whereby the approximate solution can be easily compute from the previous approximation $x_{m-1}$ and a small number of vectors that are being updated at each step. This proceduce is called DIOM. And, of course, $m$ is increasing.

In [196]:
function DIOM(A:: Matrix, b:: Vector, x0:: Vector, m_max:: Int, k:: Int, tol=1.0e-6)
    n = size(A, 1)
    V = zeros(n, m_max + 1)
    H = zeros(m_max + 1, m_max)
    P = zeros(n, m_max)
    
    r0 = b - A*x0
    beta = la.norm(r0)
    zeta = beta
    V[:, 1] = r0/beta

    println("x[0] = $x0")
    for m = 1:m_max
        w = A*V[:, m]
        # IOM block
        for i = max(1, m - k + 1):m
            H[i, m] = la.dot(w, V[:, i])
            w = w - H[i, m]*V[:, i] 
        end
        H[m+1, m] = la.norm(w)
        V[:, m+1] = w/H[m+1, m]
        # update LU factorization of Hm, LU approach
        L, U = la.lu(H[1:m, 1:m])
        if U[m ,m] < eps()
            break
        end
        zeta = m == 1 ? beta : -L[m, m-1] * zeta    
        i = m - k + 1
        sum_p = i <= 0 ? 0 : P[:, i:m-1] * U[i:m-1, m]  
        P[:, m] = 1/U[m, m] * (V[:, m] .- sum_p)
        x = x0 + zeta*P[:, m]
        println("x[$m] = $x")
        if la.norm(x - x0) < tol
            break
        end
        # updating for next step
        x0 = copy(x)
    end
end

DIOM (generic function with 2 methods)

In [197]:
m = 4
k = 2
DIOM(A, b, x0, m, k)

x[0] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
x[1] = [0.0, 1.069915254237288, 0.0, 1.2838983050847455, -0.4279661016949152, 1.2838983050847455]
x[2] = [1.0223756706686236, 1.6864518923080625, 1.0223756706686236, 2.0609195678848975, 0.8310997762432935, 2.0609195678848975]
x[3] = [0.9907832820475928, 1.99163513765398, 0.9907832820475928, 2.0053245062998073, 1.0118246060372202, 2.0053245062998073]
x[4] = [1.0, 2.0, 1.0, 1.9999999999999998, 1.0000000000000002, 1.9999999999999998]


## The Generalized Minimum Residual Method (GMRES)

In [198]:
function omega(H:: Matrix, i:: Int64)
    # Input: (m + 1) x m matrix H and column i
    # Output: (m + 1) x (m + 1) matrix O
    m = size(H, 2)
    O = 1.0 * Matrix(la.I, m + 1, m + 1) 
    hyp = sqrt(H[i, i]^2 + H[i + 1, i]^2)
    si = H[i + 1, i]/hyp
    ci = H[i, i]/hyp
    
    O[i, i] = ci
    O[i, i + 1] = si
    O[i + 1, i] = -si
    O[i + 1, i + 1] = ci
    return O 
end

omega (generic function with 1 method)

In [199]:
function givens(H:: Matrix, beta:: Vector)
    # Input: (m + 1)xm matrix H and (m + 1)x1 vector beta
    # Output: m x m upper triangular matrix R and mx1 vector beta
    m = size(H, 2)
    R = copy(H)
    for i = 1:m
        upper_omega = omega(R, i)
        beta = upper_omega * beta
        R = upper_omega * R # makes 0 (i, i+1) element of H
    end
    # delete last row of R and beta; page 176 book 1
    return R[1:m, :], beta[1:m]
end

givens (generic function with 1 method)

In [200]:
function e_vector(m:: Int64, k:: Int64)
    e = zeros(m)
    e[k] = 1.0
    return e
end

e_vector (generic function with 1 method)

In [201]:
function restarted_GMRES(A:: Matrix, b:: Vector, x0:: Vector, m:: Int64, tol=1.0e-5)
    # Solve Ax = b using the GMRES method
    # Input: n x n matrix A, n x 1 vector b,
    # initial approximation x0, integer m < n,
    # error tolerance tol
    println("x[0] = $x0")
    for i = 1:MAX_ITER
        r = b - A*x0
        # Arnoldi process
        H, V = arnoldi(A, r, m)
        beta = la.norm(r)
        # GMRES process
        # Solve the (m + 1)xm least-squares problem using Givens transformation
        R, g = givens(H, beta*e_vector(m + 1, 1))
        y = la.inv(R)*g
        x = x0 + V[:, 1:m]*y
        println("x[$i] = $x")
        if la.norm(x0 - x) < tol
            print("Number of iterations: $i")
            return nothing
        end
        x0 = copy(x)
    end
end

restarted_GMRES (generic function with 2 methods)

In [202]:
m = 4
restarted_GMRES(A, b, x0, m)

x[0] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
x[1] = [1.0000000000000002, 1.9999999999999996, 1.0000000000000002, 2.0, 1.0, 2.0]
x[2] = [0.9999999999999999, 1.9999999999999998, 0.9999999999999999, 2.0, 0.9999999999999999, 2.0]
Number of iterations: 2

In [207]:
function QGMRES(A:: Matrix, b:: Vector, x0:: Vector, m:: Int64, k:: Int64, tol=1.0e-5)
    # Solve Ax = b using the GMRES method
    # Input: n x n matrix A, n x 1 vector b,
    # initial approximation x0, integer m < n,
    # error tolerance tol
    println("x[0] = $x0")
    for i = 1:MAX_ITER
        r = b - A*x0
        # Arnoldi process
        H, V = incomplete_arnoldi(A, r, m, k)
        print("Hola")
        beta = la.norm(r)
        # GMRES process
        # Solve the (m + 1)xm least-squares problem using Givens transformation
        R, g = givens(H, beta*e_vector(m + 1, 1))
        y = la.inv(R)*g
        x = x0 + V[:, 1:m]*y
        println("x[$i] = $x")
        if la.norm(x0 - x) < tol
            print("Number of iterations: $i")
            return nothing
        end
        x0 = copy(x)
    end
end

QGMRES (generic function with 2 methods)

In [210]:
m = 3
k = 2
QGMRES(A, b, x0, m, k)

x[0] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Holax[1] = [0.9905665343243522, 1.9904398530223675, 0.9905665343243522, 2.0051849096588614, 1.0109017962460918, 2.0051849096588614]
Holax[2] = [0.9999434000739091, 1.9998868001478183, 0.9999434000739091, 1.9998868001478183, 0.999943400073909, 1.9998868001478183]
Holax[3] = [0.99999946606654, 1.9999994588963876, 0.99999946606654, 2.0000002934655035, 1.0000006170408617, 2.0000002934655035]
Holax[4] = [0.9999999967964484, 1.9999999935928967, 0.9999999967964484, 1.9999999935928967, 0.9999999967964484, 1.9999999935928967]
Number of iterations: 4

In [234]:
function DQGMRES(A:: Matrix, b:: Vector, x0:: Vector, m_max:: Int64, k::Int64, tol=1.0e-6)
    n = size(A, 1)
    V = zeros(n, m_max + 1)
    H = zeros(m_max + 1, m_max)
    P = zeros(n, n)
    gamma = zeros(m_max)

    r0 = b - A*x0
    gamma[1] = la.norm(r0)
    V[:, 1] = r0/gamma[1]

    for m = 1:m_max
        w = A*V[:, m]
        for i = max(1, m - k + 1):m
            H[i, m] = la.dot(w, V[:, i])
            w = w - H[i, m]*V[:, i] 
        end
        H[m+1, m] = la.norm(w)
        V[:, m+1] = w/H[m+1, m]

        R, g = givens(H[1:m + 1, 1:m], beta*e_vector(m + 1, 1))
        y = la.inv(R)*g
        x = x0 + V[:, 1:m]*y
        if la.norm(x - x0) < tol
            return x
        end
        # updating for next step
        x0 = copy(x)
    end
    x
end

DQGMRES (generic function with 2 methods)

In [235]:
m = 4
k = 2
DQGMRES(A, b, x0, m, k)

6-element Vector{Float64}:
 0.9999999999999999
 1.9999999999999998
 0.9999999999999999
 2.0
 0.9999999999999999
 2.0

In [ ]:
function DQGMRES(A:: Matrix, b:: Vector, x0:: Vector, tol=1.0e-6)
    n, m = size(A)
    k = 1

    V = zeros((n, m+1))
    H = zeros((m+1, m))
    P = zeros((n, n))
    gamma = zeros(m)

    r0 = b - A*x0
    gamma[1] = la.norm(r0)
    V[:, 1] = r0/gamma[1]

    for j=1:MAX_ITER
        w = A*V[:, j]
        for i = max(1, j - k + 1):j
            H[i, j] = la.dot(w, V[:, i])
            w = w - H[i, j]*V[:, i] 
        end
        H[j+1, j] = la.norm(w)
        V[:, j+1] = w/H[j+1, j]
        
        for i = m-k:m-1
            H = givens_transformation(H, i) * H
        end

        hyp = sqrt(H[j, j]^2 + H[j + 1, j]^2)
        sj = H[j + 1, j]/hyp
        cj = H[j, j]/hyp

        gamma[j + 1] = -sj*gamma[j]
        gamma[j] = cj*gamma[j]
        H[j, j] = cj*H[j, j] + sj*H[j+1, j]
        
        P[:, j] = (V[:, j] - P[:, m-k:m-1]*H[m-k:m-1, j])/H[j, j]
        x = x0 + gamma[j]*P[:, j]
        
        if la.norm(x - x0) < tol
            return x
        end
        # updating for next step
        x0 = copy(x)
    end
    x
end

DQGMRES (generic function with 4 methods)